In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=15)

import warnings
warnings.filterwarnings("ignore")

import os
os.mkdir('out') if not os.path.exists('out') else None

# import necessarry libs
from imageio import imread
from imageio import imwrite as imsave
imwrite = lambda im, fname: imsave(fname, np.maximum(0, np.minimum(255, (im*255))).astype('uint8'))
import numpy as np
from scipy.ndimage.filters import correlate, convolve
from scipy.ndimage import gaussian_filter

from skimage.color import rgb2gray
from skimage.transform import resize, rotate

# define gaussian filter
def f_gauss(size, sigma):
    # gaussian filter
    x, y = np.mgrid[-size[0]//2 + 1:size[0]//2 + 1, -size[1]//2 + 1:size[1]//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

In [ ]:
i1 = rgb2gray(imread('image52.png')/255.0)
i2 = rgb2gray(imread('image53.png')/255.0)

In [ ]:
hg = f_gauss([7, 7], 1)

origS = i1.shape

# masque
m = np.zeros(origS)
m[:, 0:128] = 1

resizeFcn = lambda im: resize(im, origS, order=0)  # order=0 --> Nearest-neighbor

In [ ]:
# combinaison
for i in range(4):
    i1f = correlate(i1, hg, mode='reflect');
    i2f = correlate(i2, hg, mode='reflect');
    mf = correlate(m, hg, mode='reflect');

    c = (i1-i1f) * mf + (i2-i2f) * (1-mf)

    imwrite(resizeFcn(c+.5), 'out/stage%02d.jpg'%i)
    imwrite(resizeFcn(i1-i1f+.5), 'out/left%02d.jpg'%i)
    imwrite(resizeFcn(i2-i2f+.5), 'out/right%02d.jpg'%i)
    imwrite(resizeFcn(mf), 'out/mask%02d.jpg'%i)

    imwrite(resizeFcn(i1), 'out/leftF%02d.jpg'%i)
    imwrite(resizeFcn(i2), 'out/rightF%02d.jpg'%i)

    # downsample
    # i1 = i1 - i1f
    # i2 = i2 - i2f
    i1 = i1f[0::2, 0::2]
    i2 = i2f[0::2, 0::2]
    m = mf[0::2, 0::2]

In [ ]:
i = 4  # i + 1
imStageNext = i1f * mf + i2f * (1 - mf)
imwrite(resizeFcn(imStageNext), 'out/stage%02d.jpg'%i)
imwrite(resizeFcn(i1f), 'out/left%02d.jpg'%i)
imwrite(resizeFcn(i2f), 'out/right%02d.jpg'%i)
imwrite(resizeFcn(mf), 'out/mask%02d.jpg'%i)

In [ ]:
# we can run this cell once, else re-compute the imStageNext in the above cell
for i in reversed(range(4)):
    imStageCur = imread('out/stage%02d.jpg'%i)/255.0-.5
    imStageCur = resize(imStageCur, np.asarray(imStageNext.shape)*2, order=0)
    # imStageCur[1::2,1::2] = 0
    
    # upsample
    imStageNext = resize(imStageNext, imStageCur.shape, order=0)
    # imStageNext[1::2,1::2] = 0
    imStageNext = correlate(imStageNext, hg, mode='reflect')
    
    imStageNext = imStageNext + imStageCur
    imwrite(resizeFcn(imStageNext), 'out/recon%02d.jpg'%i)